# CitiBike Data

## Overview

#### This notebook contains the python code to gather the citibike data files for 9/2016 - 9/2022.  The data is combined into one dataframe, cleaned up and saved a one CSV to be used for a Tableau dashboard and story.

In [1]:
import pandas as pd
import re
import os
import json

## Extract

* The CitiBike csv files are loaded into a pandas dataframe using pd.read_csv
    * Create a dictionary of the files in the raw files directory
    * Loop through the dictionary
        * Load the CSV for the neighborhood into a temporary dataframe
        * Add columns to hold the neighborhood and demographic category (age, income, education)
        * Append the temporary dataframe to the consolidated dataframe

In [2]:
#### ----  Extract New Format csvs ---- ####
NewFormat_df = pd.DataFrame()

#Loop through the files in the directory
for file in os.listdir("./Resources/RawFiles_NewFormat"):
    #Load the current csv to temporary dataframe
    citibike_csv = f"./Resources/RawFiles_NewFormat/{file}"
    temp_citibike_df = pd.read_csv(citibike_csv)
    
    #Append the neighborhood dataframe to the age dataframe
    NewFormat_df = pd.concat([NewFormat_df, temp_citibike_df], ignore_index = True)

NewFormat_df


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,74A4206E7487CBC9,docked_bike,2021-05-23 16:51:00,2021-05-23 18:38:21,9 St HBLR - Jackson St & 8 St,HB305,9 St HBLR - Jackson St & 8 St,HB305,40.747907,-74.038411,40.747907,-74.038412,casual
1,58EEE2950FFE01CE,docked_bike,2021-05-31 16:54:47,2021-05-31 16:55:28,9 St HBLR - Jackson St & 8 St,HB305,9 St HBLR - Jackson St & 8 St,HB305,40.747907,-74.038411,40.747907,-74.038412,member
2,1429D912C16EEE59,docked_bike,2021-05-25 16:19:34,2021-05-25 17:03:06,9 St HBLR - Jackson St & 8 St,HB305,9 St HBLR - Jackson St & 8 St,HB305,40.747907,-74.038411,40.747907,-74.038412,casual
3,FE9C5B74167CBCCD,docked_bike,2021-05-22 17:32:19,2021-05-22 17:41:27,9 St HBLR - Jackson St & 8 St,HB305,Grand St & 2 St,HB405,40.747907,-74.038411,40.739130,-74.036180,casual
4,B88D37626F000BBA,docked_bike,2021-05-14 09:48:34,2021-05-14 10:17:36,Union St,JC051,Newark Ave,JC032,40.718211,-74.083639,40.721525,-74.046305,casual
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1317447,3F8FD9A051C0E3DA,docked_bike,2021-02-24 18:05:45,2021-02-24 18:12:27,Columbus Drive,JC014,Newport Pkwy,JC008,40.718355,-74.038914,40.728745,-74.032108,member
1317448,B91F3593C0C06D04,docked_bike,2021-02-22 18:22:04,2021-02-22 18:28:45,Columbus Drive,JC014,Newport Pkwy,JC008,40.718355,-74.038914,40.728745,-74.032108,member
1317449,A790954A017D07CB,docked_bike,2021-02-13 12:21:49,2021-02-13 12:27:51,Columbus Drive,JC014,Newport Pkwy,JC008,40.718355,-74.038914,40.728745,-74.032108,member
1317450,42B1932D06CEE034,docked_bike,2021-02-25 11:20:29,2021-02-25 11:25:23,Columbus Drive,JC014,Morris Canal,JC072,40.718355,-74.038914,40.712419,-74.038526,member


In [3]:
#### ----  Extract Old Format csvs with lower case headings ---- ####
OldFormat_df = pd.DataFrame()

#Loop through the files in the directory
for file in os.listdir("./Resources/RawFiles_OldFormat"):
    #Load the current csv to temporary dataframe
    try:
        citibike_csv = f"./Resources/RawFiles_OldFormat/{file}"
        temp_citibike_df = pd.read_csv(citibike_csv)
    except:
        print(f"{file} is throwing an error")
    
    #Append the neighborhood dataframe to the age dataframe
    OldFormat_df = pd.concat([OldFormat_df, temp_citibike_df], ignore_index = True)

OldFormat_df

.DS_Store is throwing an error


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,156,2020-10-01 00:02:40.2600,2020-10-01 00:05:17.0140,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,42293,Subscriber,1996.0,1
1,143,2020-10-01 00:02:53.5020,2020-10-01 00:05:16.9650,3186,Grove St PATH,40.719586,-74.043117,3270,Jersey & 6th St,40.725289,-74.045572,44740,Subscriber,1998.0,1
2,305,2020-10-01 00:04:31.3160,2020-10-01 00:09:36.3840,3273,Manila & 1st,40.721651,-74.042884,3269,Brunswick & 6th,40.726012,-74.050389,41369,Subscriber,1988.0,2
3,1097,2020-10-01 00:05:18.3680,2020-10-01 00:23:36.1240,3199,Newport Pkwy,40.728745,-74.032108,3281,Leonard Gordon Park,40.745910,-74.057271,42285,Customer,2001.0,1
4,277,2020-10-01 00:09:17.5990,2020-10-01 00:13:55.5150,3199,Newport Pkwy,40.728745,-74.032108,3638,Washington St,40.724294,-74.035483,45231,Subscriber,1965.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402057,270,2020-10-13 15:05:35.0860,2020-10-13 15:10:05.3430,3207,Oakland Ave,40.737604,-74.052478,3640,Journal Square,40.733670,-74.062500,44744,Subscriber,1963.0,2
1402058,400,2020-10-13 15:09:03.4890,2020-10-13 15:15:43.9750,3209,Brunswick St,40.724176,-74.050656,3209,Brunswick St,40.724176,-74.050656,45345,Subscriber,1984.0,1
1402059,206,2020-10-13 15:11:34.3500,2020-10-13 15:15:00.5030,3195,Sip Ave,40.730897,-74.063913,3194,McGinley Square,40.725340,-74.067622,47019,Subscriber,1993.0,1
1402060,216,2020-10-13 15:11:49.1510,2020-10-13 15:15:25.6930,3195,Sip Ave,40.730897,-74.063913,3225,Baldwin at Montgomery,40.723659,-74.064194,42191,Subscriber,1966.0,1


In [4]:
#### ----  Extract Old Format csvs with upper case headings ---- ####
OldFormat_UC_df = pd.DataFrame()

#Loop through the files in the directory
for file in os.listdir("./Resources/RawFiles_OF_UpperC"):
    #Load the current csv to temporary dataframe
    try:
        citibike_csv = f"./Resources/RawFiles_OF_UpperC/{file}"
        temp_citibike_df = pd.read_csv(citibike_csv)
    except:
        print(f"{file} is throwing an error")
    
    #Append the neighborhood dataframe to the age dataframe
    OldFormat_UC_df = pd.concat([OldFormat_UC_df, temp_citibike_df], ignore_index = True)

OldFormat_UC_df

,Trip Duration,Start Time,Stop Time,Start Station ID,Start Station Name,Start Station Latitude,Start Station Longitude,End Station ID,End Station Name,End Station Latitude,End Station Longitude,Bike ID,User Type,Birth Year,Gender
0,148,2017-01-01 00:21:32,2017-01-01 00:24:01,3276,Marin Light Rail,40.714584,-74.042817,3185,City Hall,40.717732,-74.043845,24575,Subscriber,1983.0,1
1,1283,2017-01-01 00:24:35,2017-01-01 00:45:58,3183,Exchange Place,40.716247,-74.033459,3198,Heights Elevator,40.748716,-74.040443,24723,Subscriber,1978.0,1
2,372,2017-01-01 00:38:19,2017-01-01 00:44:31,3183,Exchange Place,40.716247,-74.033459,3211,Newark Ave,40.721525,-74.046305,24620,Subscriber,1989.0,1
3,1513,2017-01-01 00:38:37,2017-01-01 01:03:50,3194,McGinley Square,40.725340,-74.067622,3271,Danforth Light Rail,40.692640,-74.088012,24668,Subscriber,1961.0,1
4,639,2017-01-01 01:47:52,2017-01-01 01:58:31,3183,Exchange Place,40.716247,-74.033459,3203,Hamilton Park,40.727596,-74.044247,26167,Subscriber,1993.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139130,422,2016-11-30 22:17:45,2016-11-30 22:24:48,3185,City Hall,40.717732,-74.043845,3214,Essex Light Rail,40.712774,-74.036486,24530,Subscriber,1961.0,1
139131,105,2016-11-30 22:38:35,2016-11-30 22:40:20,3186,Grove St PATH,40.719586,-74.043117,3185,City Hall,40.717732,-74.043845,24556,Subscriber,1957.0,1
139132,150,2016-11-30 23:02:50,2016-11-30 23:05:20,3186,Grove St PATH,40.719586,-74.043117,3272,Jersey & 3rd,40.723332,-74.045953,24681,Subscriber,1952.0,1
139133,142,2016-11-30 23:03:04,2016-11-30 23:05:27,3186,Grove St PATH,40.719586,-74.043117,3213,Van Vorst Park,40.718489,-74.047727,24620,Subscriber,1989.0,1


## Transform

#### Transform the CitiBike data by:
* Dropping rows with null values
* Combining the 3 dataframes by standardizing them

In [10]:
# Reduce the columns of the Crime Data to neighborhood, occurred_date, offense_category, offense, latitude, longitude, crime_count
reduced_OldFormat_UC = OldFormat_UC_df[['Start Time', 'Stop Time', 'Start Station Name', 'Start Station Latitude', 'Start Station Longitude', 'End Station Name', 'End Station Latitude', 'End Station Longitude', 'User Type']]

reduced_OldFormat_LC = OldFormat_df[['starttime', 'stoptime', 'start station name', 'start station latitude', 'start station longitude', 'end station name', 'end station latitude', 'end station longitude', 'usertype']]

CitiBike_df = NewFormat_df[['started_at', 'ended_at', 'start_station_name', 'start_lat', 'start_lng', 'end_station_name', 'end_lat', 'end_lng', 'member_casual']]

# Rename the lower case old format column headers
OldFormat_LC_transformed = reduced_OldFormat_LC.rename(columns={"starttime": "started_at",
                                                          "stoptime": "ended_at",
                                                          "start station name": "start_station_name",
                                                          "start station latitude": "start_lat",
                                                          "start station longitude": "start_lng",
                                                          "end station name": "end_station_name",
                                                          "end station latitude": "end_lat",
                                                          "end station longitude": "end_lng",
                                                          "usertype": "member_casual"})

# Rename the upper case old format column headers
OldFormat_UC_transformed = reduced_OldFormat_UC.rename(columns={"Start Time": "started_at",
                                                          "Stop Time": "ended_at",
                                                          "Start Station Name": "start_station_name",
                                                          "Start Station Latitude": "start_lat",
                                                          "Start Station Longitude": "start_lng",
                                                          "End Station Name": "end_station_name",
                                                          "End Station Latitude": "end_lat",
                                                          "End Station Longitude": "end_lng",
                                                          "User Type": "member_casual"})

# Combine all 3 datasets
CitiBike_df = pd.concat([CitiBike_df, OldFormat_LC_transformed], ignore_index = True)
CitiBike_df = pd.concat([CitiBike_df, OldFormat_UC_transformed], ignore_index = True)

# Eliminate Records with no start time
CitiBike_df = CitiBike_df[CitiBike_df['started_at'].notna()]

# Remove rows with no end time
CitiBike_df = CitiBike_df[CitiBike_df['ended_at'].notna()]

# Replace values in a column
CitiBike_df['member_casual'] = CitiBike_df['member_casual'].replace(['Customer', 'Subscriber'], ['casual', 'member'])

#shorten Lat and lng so they are consistent for each station
CitiBike_df['start_lat'] = CitiBike_df['start_lat'].round(decimals = 4)
CitiBike_df['start_lng'] = CitiBike_df['start_lng'].round(decimals = 4)
CitiBike_df['end_lat'] = CitiBike_df['end_lat'].round(decimals = 4)
CitiBike_df['end_lng'] = CitiBike_df['end_lng'].round(decimals = 4)

#Create Time Duration field
CitiBike_df['started_at'] = pd.to_datetime(CitiBike_df['started_at'])
CitiBike_df['ended_at'] = pd.to_datetime(CitiBike_df['ended_at'])
CitiBike_df['time_duration'] = CitiBike_df['ended_at']-CitiBike_df['started_at']

CitiBike_df


,started_at,ended_at,start_station_name,start_lat,start_lng,end_station_name,end_lat,end_lng,member_casual,time_duration
0,2021-05-23 16:51:00,2021-05-23 18:38:21,9 St HBLR - Jackson St & 8 St,40.7479,-74.0384,9 St HBLR - Jackson St & 8 St,40.7479,-74.0384,casual,0 days 01:47:21
1,2021-05-31 16:54:47,2021-05-31 16:55:28,9 St HBLR - Jackson St & 8 St,40.7479,-74.0384,9 St HBLR - Jackson St & 8 St,40.7479,-74.0384,member,0 days 00:00:41
2,2021-05-25 16:19:34,2021-05-25 17:03:06,9 St HBLR - Jackson St & 8 St,40.7479,-74.0384,9 St HBLR - Jackson St & 8 St,40.7479,-74.0384,casual,0 days 00:43:32
3,2021-05-22 17:32:19,2021-05-22 17:41:27,9 St HBLR - Jackson St & 8 St,40.7479,-74.0384,Grand St & 2 St,40.7391,-74.0362,casual,0 days 00:09:08
4,2021-05-14 09:48:34,2021-05-14 10:17:36,Union St,40.7182,-74.0836,Newark Ave,40.7215,-74.0463,casual,0 days 00:29:02
...,...,...,...,...,...,...,...,...,...,...
2858644,2016-11-30 22:17:45,2016-11-30 22:24:48,City Hall,40.7177,-74.0438,Essex Light Rail,40.7128,-74.0365,member,0 days 00:07:03
2858645,2016-11-30 22:38:35,2016-11-30 22:40:20,Grove St PATH,40.7196,-74.0431,City Hall,40.7177,-74.0438,member,0 days 00:01:45
2858646,2016-11-30 23:02:50,2016-11-30 23:05:20,Grove St PATH,40.7196,-74.0431,Jersey & 3rd,40.7233,-74.0460,member,0 days 00:02:30
2858647,2016-11-30 23:03:04,2016-11-30 23:05:27,Grove St PATH,40.7196,-74.0431,Van Vorst Park,40.7185,-74.0477,member,0 days 00:02:23


## Load 
##### Save the concatonated infomation to new csv.

In [11]:
#Export to new file
CitiBike_df.to_csv('Resources/CitiBike_consolidated.csv', index=True)

In [12]:
#Export 20% random sample to new file for dev and test
CitiBike_sample_df = CitiBike_df.sample(frac=0.20)
CitiBike_sample_df.to_csv('Resources/CitiBike_sample.csv', index=True)